In [1]:
import spikeinterface as si
import matplotlib.pyplot as plt
%matplotlib inline
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import numpy as np
import pandas as pd
import seaborn as sns

/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_trigger_time_range(file_path):
    recording = se.read_blackrock(file_path=file_path)
    recording = recording.channel_slice(channel_ids=['98'])
    trace_mat = recording.get_traces(start_frame=  1,
                                    end_frame= recording.get_duration() * 10000,
                                    channel_ids= ['98'])
    trace_mat = trace_mat.astype(int)
    trace_mat[trace_mat < 20000] = 0
    trace_mat[trace_mat >= 20000] = 1

    starts = []
    ends = []

    for i in range(1, len(trace_mat)):
        if trace_mat[i] == 1 and trace_mat[i-1] == 0:
            starts.append(i+1)
        elif trace_mat[i] == 0 and trace_mat[i-1] == 1:
            ends.append(i)

    df = pd.DataFrame({'start': starts, 'end': ends})
    df = df[2:-2]
    df['end'] = df['start'] + 5000
    date = file_path.split("/")[8]
    date = date.split("_")[1]
    df['date'] = date
    return df


In [3]:
import os
recording_dir = os.listdir("/media/ubuntu/sda/data/mouse6/ns4/natural_image/")
trigger_time = pd.DataFrame()
recording_dir.remove("mouse6_102122_natural_image_001.ns4")

for recording in recording_dir:
    temp = get_trigger_time_range(f'/media/ubuntu/sda/data/mouse6/ns4/natural_image/{recording}')
    trigger_time = pd.concat((trigger_time, temp), axis = 0)

In [4]:
for date in trigger_time['date'].unique():
    temp = trigger_time[trigger_time['date'] == date]
    print(f'{date}: {len(temp)/117}')

022223: 10.0
112022: 10.0
082322: 10.0
032123: 10.0
122022: 10.0
092422: 10.0
012123: 10.0
042323: 10.0
072322: 10.0
022522: 10.0
021322: 15.0
052422: 10.0
042422: 15.0
062422: 10.0
031722: 10.0


- 102122 recording不全

In [4]:
trigger_time = pd.DataFrame()

recording = se.read_blackrock(file_path="/media/ubuntu/sda/data/mouse6/ns4/natural_image/mouse6_102122_natural_image_001.ns4")
recording = recording.channel_slice(channel_ids=['98'])

trace_mat = recording.get_traces(start_frame=  1,
                                end_frame= recording.get_duration() * 10000,
                                channel_ids= ['98'])
trace_mat = trace_mat.astype(int)
trace_mat[trace_mat < 20000] = 0
trace_mat[trace_mat >= 20000] = 1

starts = []
ends = []

for i in range(1, len(trace_mat)):
    if trace_mat[i] == 1 and trace_mat[i-1] == 0:
        starts.append(i+1)
    elif trace_mat[i] == 0 and trace_mat[i-1] == 1:
        ends.append(i)

df = pd.DataFrame({'start': starts, 'end': ends})
df = df[2:-2]
df = df[:1053]
df['end'] = df['start'] + 5000
date = '102122'
df['date'] = date
trigger_time = pd.concat((trigger_time, df), axis = 0)

###### Trigger order

In [6]:
def get_image_order(file_path):
    from scipy.io import loadmat
    mat_data = loadmat(file_path)
    mat_data = mat_data['Image_show_indices_natural']
    mat_data_pure = mat_data.ravel()

    row_indices = np.repeat(np.arange(mat_data.shape[0]), mat_data.shape[1])
    suffixes = np.array([f"_{i+1}" for i in row_indices])
    mat_data_order = np.array([f"{val}{suffix}" for val, suffix in zip(mat_data_pure, suffixes)])
    return mat_data_pure, mat_data_order

In [7]:
image_order_dict = {}
image_order_dict['021322'] = {}
image_order_dict['022522'] = {}
image_order_dict['031722'] = {}
image_order_dict['042422'] = {}
image_order_dict['052422'] = {}
image_order_dict['062422'] = {}
image_order_dict['072322'] = {}
image_order_dict['082322'] = {}
image_order_dict['092422'] = {}
image_order_dict['102122'] = {}
image_order_dict['112022'] = {}
image_order_dict['122022'] = {}
image_order_dict['012123'] = {}
image_order_dict['022223'] = {}
image_order_dict['032123'] = {}
image_order_dict['042323'] = {}

image_order_dict['021322']['pure'], image_order_dict['021322']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_021322/Image_StmulationIndices.mat")
image_order_dict['022522']['pure'], image_order_dict['022522']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_022522/Image_StmulationIndices.mat")
image_order_dict['031722']['pure'], image_order_dict['031722']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_031722/Image_StmulationIndices.mat")
image_order_dict['042422']['pure'], image_order_dict['042422']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_042422/Image_StmulationIndices.mat")
image_order_dict['052422']['pure'], image_order_dict['052422']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_052422/Image_StmulationIndices.mat")
image_order_dict['062422']['pure'], image_order_dict['062422']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_062422/Image_StmulationIndices.mat")
image_order_dict['072322']['pure'], image_order_dict['072322']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_072322/Image_StmulationIndices.mat")
image_order_dict['082322']['pure'], image_order_dict['082322']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_082322/Image_StmulationIndices.mat")
image_order_dict['092422']['pure'], image_order_dict['092422']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_092422/Image_StmulationIndices.mat")
image_order_dict['102122']['pure'], image_order_dict['102122']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_102122/Image_StmulationIndices.mat")
image_order_dict['112022']['pure'], image_order_dict['112022']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_112022/Image_StmulationIndices.mat")
image_order_dict['122022']['pure'], image_order_dict['122022']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_122022/Image_StmulationIndices.mat")
image_order_dict['012123']['pure'], image_order_dict['012123']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_012123/Image_StmulationIndices.mat")
image_order_dict['022223']['pure'], image_order_dict['022223']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_022223/Image_StmulationIndices.mat")
image_order_dict['032123']['pure'], image_order_dict['032123']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_032123/Image_StmulationIndices.mat")
image_order_dict['042323']['pure'], image_order_dict['042323']['order'] = get_image_order("/media/ubuntu/sda/data/trigger/mouse6/mouse6_042323/Image_StmulationIndices.mat")

In [8]:
trigger_time['image'] = None
trigger_time['order'] = None
for date in trigger_time['date'].unique():
    if date != '102122':
        trigger_time.loc[trigger_time['date'] == date, 'image'] = image_order_dict[date]['pure']
        trigger_time.loc[trigger_time['date'] == date, 'order'] = image_order_dict[date]['order']
    else:
        trigger_time.loc[trigger_time['date'] == date, 'image'] = image_order_dict[date]['pure'][0:1053]
        trigger_time.loc[trigger_time['date'] == date, 'order'] = image_order_dict[date]['order'][0:1053]



In [9]:
trigger_time.to_csv('trigger_time.tsv', sep = '\t')
